In [1]:
%matplotlib inline


# Topic extraction with Non-negative Matrix Factorization and Latent Dirichlet Allocation


This is an example of applying :class:`sklearn.decomposition.NMF` and
:class:`sklearn.decomposition.LatentDirichletAllocation` on a corpus
of documents and extract additive models of the topic structure of the
corpus.  The output is a list of topics, each represented as a list of
terms (weights are not shown).

Non-negative Matrix Factorization is applied with two different objective
functions: the Frobenius norm, and the generalized Kullback-Leibler divergence.
The latter is equivalent to Probabilistic Latent Semantic Indexing.

The default parameters (n_samples / n_features / n_components) should make
the example runnable in a couple of tens of seconds. You can try to
increase the dimensions of the problem, but be aware that the time
complexity is polynomial in NMF. In LDA, the time complexity is
proportional to (n_samples * iterations).




In [30]:
from os import listdir
from os.path import isfile, join
from os import environ
import os
from nltk.tokenize import TreebankWordTokenizer, word_tokenize
import re
import indexes
from time import time

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import langid
import numpy as np
rng = np.random.RandomState(10102016)
np.random.seed(18101995)

In [6]:
#sent_text = nltk.sent_tokenize(text) # this gives us a list of sentences
with open( os.path.join(os.environ["AAN_DIR"],"papers_text/{0}".format("W04-0203.txt"))) as f:
    good_text  = f.read().replace('-\n','')
with open( os.path.join(os.environ["AAN_DIR"],"papers_text/{0}".format("W04-0207.txt"))) as f:
    good_text2  = f.read().replace('-\n','')
with open( os.path.join(os.environ["AAN_DIR"],"papers_text/{0}".format("W04-0205.txt"))) as f:
    bad_text  = f.read().replace('-\n','')
with open( os.path.join(os.environ["AAN_DIR"],"papers_text/{0}".format("J79-1011.txt"))) as f:
    bad_text2 = f.read().replace('-\n','')   
    

In [7]:
from indexes import ACL_metadata
acl = ACL_metadata()

In [8]:
acl.df

,authors,genders,title,venue,year
id,,,,,
E03-1062,"[Piwek, Paul]",[Gender.male],A Flexible Pragmatics-Driven Language Generato...,EACL,2003
E06-1001,"[McConville, Mark]",[Gender.male],Inheritance And The CCG Lexicon,EACL,2006
E06-1002,"[Bunescu, Razvan C., Pa&scedil;ca, Marius]","[Gender.male, Gender.male]",Using Encyclopedic Knowledge For Named Entity ...,EACL,2006
E06-1003,"[Tanev, Hristo, Magnini, Bernardo]","[Gender.male, Gender.male]",Weakly Supervised Approaches For Ontology Popu...,EACL,2006
E06-1004,"[Udupa, Raghavendra, Maji, Hemanta K.]","[Gender.male, Gender.male]",Computational Complexity Of Statistical Machin...,EACL,2006
E06-1005,"[Matusov, Evgeny, Ueffing, Nicola, Ney, Hermann]","[Gender.male, Gender.female, Gender.male]",Computing Consensus Translation For Multiple M...,EACL,2006
E06-1006,"[Yang, Mei, Kirchhoff, Katrin]","[Gender.female, Gender.female]",Phrase-Based Backoff Models For Machine Transl...,EACL,2006
E06-1007,"[M&uuml;ller, Christof E.]",[Gender.male],Automatic Detection Of Nonreferential It In Sp...,EACL,2006
E06-1008,"[Jonson, Rebecca]",[Gender.female],Generating Statistical Language Models From In...,EACL,2006


In [10]:
len(acl.train_files)

22405

In [12]:
def dehyphenate(s):
    return s.replace('-\n','').lower()

In [24]:
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, 
                                input='filename',
                                min_df=5,
                                #max_features=n_features,
                                stop_words='english',
                                token_pattern=r"(?u)\b[a-zA-Z][a-zA-Z]+\b",
                                preprocessor = dehyphenate
                                #tokenizer=TreebankWordTokenizer().tokenize
                                )
t0 = time()
tf = tf_vectorizer.fit_transform(acl.train_files)
print("done in %0.3fs." % (time() - t0))
print()

Extracting tf features for LDA...
done in 69.230s.



In [28]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [32]:
n_features = 1000
n_components = 10
n_top_words = 20

In [31]:
print("Fitting LDA models with tf features, "
      "and n_features=%d..."
      % (n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=20,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=rng)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))


NameError: name 'n_features' is not defined

In [173]:
print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)


Topics in LDA model:
Topic #0: semantic information representation pp features parses linguistic node np linguistics processing corpus analysis query task using distance predicate possible paper
Topic #1: scope disambiguation proceedings pages approach local based mention semantic mentions paper context data words parses using example joint language linguistic
Topic #2: semantic token predicate representation words np information using word data phrase used number base set results role features pp argument
Topic #3: features entity speech model figure semantic pages et al input integration using proper proceedings coreference type set scope word feature
Topic #4: linguistic information meaning processing sense domain semantic category world way features point context illustrates using model language words discourse tag
Topic #5: semantic scope mention features word words entity using al disambiguation mentions et information approach model set pages language proceedings data
Topic #6:

In [1]:
# Author: Olivier Grisel <olivier.grisel@ensta.org>
#         Lars Buitinck
#         Chyi-Kwei Yau <chyikwei.yau@gmail.com>
# License: BSD 3 clause

from __future__ import print_function
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

n_samples = 2000
n_features = 1000
n_components = 10
n_top_words = 20


def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()


# Load the 20 newsgroups dataset and vectorize it. We use a few heuristics
# to filter out useless terms early on: the posts are stripped of headers,
# footers and quoted replies, and common English words, words occurring in
# only one document or in at least 95% of the documents are removed.

print("Loading dataset...")
t0 = time()
dataset = fetch_20newsgroups(shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'))
data_samples = dataset.data[:n_samples]
print("done in %0.3fs." % (time() - t0))

# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))
print()

# Fit the NMF model
print("Fitting the NMF model (Frobenius norm) with tf-idf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

# Fit the NMF model
print("Fitting the NMF model (generalized Kullback-Leibler divergence) with "
      "tf-idf features, n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Loading dataset...
done in 17.985s.
Extracting tf-idf features for NMF...
done in 0.570s.
Extracting tf features for LDA...
done in 0.608s.

Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=2000 and n_features=1000...
done in 1.228s.

Topics in NMF model (Frobenius norm):
Topic #0: just people don think like know time good make way really say right ve want did ll new use years
Topic #1: windows use dos using window program os drivers application help software pc running ms screen files version card code work
Topic #2: god jesus bible faith christian christ christians does heaven sin believe lord life church mary atheism belief human love religion
Topic #3: thanks know does mail advance hi info interested email anybody looking card help like appreciated information send list video need
Topic #4: car cars tires miles 00 new engine insurance price condition oil power speed good 000 brake year models used bought
Topic #5: edu soon com send university internet mit ftp 

KeyboardInterrupt: 